# Openai Assistant API Functions

In [47]:
import openai

In [48]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [49]:
import os

In [50]:
client = openai.OpenAI(
    api_key=os.getenv('OPENAI_API_KEY')
)

In [51]:
assistants = client.beta.assistants.list()

print(assistants.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "asst_k5aovsoQxn9ckp9Y3ea7koLv",
      "created_at": 1702185964,
      "description": null,
      "file_ids": [],
      "instructions": "You are a helpful assistant. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.",
      "metadata": {},
      "model": "gpt-4-1106-preview",
      "name": "YouTube-Functions-Demo",
      "object": "assistant",
      "tools": [
        {
          "function": {
            "name": "get_weather",
            "parameters": {
              "type": "object",
              "properties": {
                "location": {
                  "type": "string",
                  "description": "The city and state e.g. San Francisco, CA"
                },
                "unit": {
                  "type": "string",
                  "enum": [
                    "c",
                    "f"
                  ],
                  "description": "The temperature unit 

# Create Assistant

In [52]:
tools = [
    {
        "function": {
            "name": "get_weather",
            "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state e.g. San Francisco, CA"
                        },
                        "unit": {
                            "type": "string",
                            "enum": [
                                "c",
                                "f"
                            ],
                            "description": "The temperature unit to use. Infer this from the user's location."
                        }
                    },
                "required": ["location"]
            },
            "description": "Get the current weather"
        },
        "type": "function"
    },
    {
        "function": {
            "name": "get_n_day_weather_forecast",
            "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state e.g. San Francisco, CA"
                        },
                        "unit": {
                            "type": "string",
                            "enum": [
                                "c",
                                "f"
                            ]
                        },
                        "num_days": {
                            "type": "integer",
                            "description": "The number of days to forecast"
                        }
                    },
                "required": ["location", "num_days"]
            },
            "description": "Get an N-day weather forecast"
        },
        "type": "function"
    }
]

In [53]:
weather_helper = client.beta.assistants.create(
    model='gpt-4-1106-preview',
    name='YouTube-Functions-Demo-From-Notebook',
    instructions="You are a helpful assistant. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.",
    tools=tools,
    description='This is an assistant created from a notebook.'
)

In [55]:
print(weather_helper.model_dump_json(indent=2))

{
  "id": "asst_wYPfnomOOdUnvwjccQ7PSnz1",
  "created_at": 1702193873,
  "description": "This is an assistant created from a notebook.",
  "file_ids": [],
  "instructions": "You are a helpful assistant. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.",
  "metadata": {},
  "model": "gpt-4-1106-preview",
  "name": "YouTube-Functions-Demo-From-Notebook",
  "object": "assistant",
  "tools": [
    {
      "function": {
        "name": "get_weather",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state e.g. San Francisco, CA"
            },
            "unit": {
              "type": "string",
              "enum": [
                "c",
                "f"
              ],
              "description": "The temperature unit to use. Infer this from the user's location."
            }
         

In [56]:
thread = client.beta.threads.create()

In [57]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What is the weather today in Ahmedabad, Gujarat?"
)

In [58]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=weather_helper.id,
  instructions="Please address the user as Raj."
)

In [59]:
retrieve_run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

In [60]:
print(retrieve_run.model_dump_json(indent=2))

{
  "id": "run_duFnRM6UzBcbEnW8eVtkl2bf",
  "assistant_id": "asst_wYPfnomOOdUnvwjccQ7PSnz1",
  "cancelled_at": null,
  "completed_at": null,
  "created_at": 1702194001,
  "expires_at": 1702194601,
  "failed_at": null,
  "file_ids": [],
  "instructions": "Please address the user as Raj.",
  "last_error": null,
  "metadata": {},
  "model": "gpt-4-1106-preview",
  "object": "thread.run",
  "required_action": {
    "submit_tool_outputs": {
      "tool_calls": [
        {
          "id": "call_nxVahCbiHLv0Ed7fBARh1M1w",
          "function": {
            "arguments": "{\"location\":\"Ahmedabad, Gujarat\"}",
            "name": "get_weather"
          },
          "type": "function"
        }
      ]
    },
    "type": "submit_tool_outputs"
  },
  "started_at": 1702194001,
  "status": "requires_action",
  "thread_id": "thread_e1fUU9V1SnLZTP0RYBdzTceo",
  "tools": [
    {
      "function": {
        "name": "get_weather",
        "parameters": {
          "type": "object",
          "propert

In [61]:
client.beta.threads.runs.submit_tool_outputs(
    run_id=run.id,
    thread_id=thread.id,
    tool_outputs=[
      {
        "tool_call_id": 'call_nxVahCbiHLv0Ed7fBARh1M1w',
        "output": "it is 29 degree Celcius",
      }
    ]
)

Run(id='run_duFnRM6UzBcbEnW8eVtkl2bf', assistant_id='asst_wYPfnomOOdUnvwjccQ7PSnz1', cancelled_at=None, completed_at=None, created_at=1702194001, expires_at=1702194601, failed_at=None, file_ids=[], instructions='Please address the user as Raj.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=1702194001, status='queued', thread_id='thread_e1fUU9V1SnLZTP0RYBdzTceo', tools=[ToolAssistantToolsFunction(function=FunctionDefinition(name='get_weather', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state e.g. San Francisco, CA'}, 'unit': {'type': 'string', 'enum': ['c', 'f'], 'description': "The temperature unit to use. Infer this from the user's location."}}, 'required': ['location']}, description='Get the current weather'), type='function'), ToolAssistantToolsFunction(function=FunctionDefinition(name='get_n_day_weather_forecast', parameters={'type': 'object', 'properties

In [62]:
retrieve_run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

In [63]:
print(retrieve_run.model_dump_json(indent=2))

{
  "id": "run_duFnRM6UzBcbEnW8eVtkl2bf",
  "assistant_id": "asst_wYPfnomOOdUnvwjccQ7PSnz1",
  "cancelled_at": null,
  "completed_at": 1702194059,
  "created_at": 1702194001,
  "expires_at": null,
  "failed_at": null,
  "file_ids": [],
  "instructions": "Please address the user as Raj.",
  "last_error": null,
  "metadata": {},
  "model": "gpt-4-1106-preview",
  "object": "thread.run",
  "required_action": null,
  "started_at": 1702194058,
  "status": "completed",
  "thread_id": "thread_e1fUU9V1SnLZTP0RYBdzTceo",
  "tools": [
    {
      "function": {
        "name": "get_weather",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state e.g. San Francisco, CA"
            },
            "unit": {
              "type": "string",
              "enum": [
                "c",
                "f"
              ],
              "description": "The temperature unit t

In [64]:
thread_messages = client.beta.threads.messages.list(thread.id)

In [65]:
user = ''
assistant = ''

for tm in reversed(thread_messages.data):
    content = tm.content[0].text.value
    if tm.role == 'user':
        user += content
    if tm.role == 'assistant':
        assistant += content

In [66]:
print(user)

What is the weather today in Ahmedabad, Gujarat?


In [67]:
print(assistant)

Today in Ahmedabad, Gujarat, the weather is currently 29 degrees Celsius.
